In [ ]:
import itertools
import pprint
import sys

In [ ]:
sys.path.append("../src")
import hw_loading
import processor_utils
from processor_utils import units
from processor_utils.units import FuncUnit
import program_utils
import sim_services
from str_utils import ICaseString

In [ ]:
capabilities = [ICaseString(cap) for cap in ["ALU", "MEM"]]
fetch_unit, decode_unit, execute_unit, memory_unit, writeback_unit = (units.UnitModel(ICaseString(name), 1, capabilities, units.LockInfo(rd_lock, wr_lock), mem_acl) for name, rd_lock, wr_lock, mem_acl in [('F', False, False, capabilities), ('D', True, False, []), ('X', False, False, []), ('M', False, False, capabilities), ('W', False, True, [])])

In [ ]:
proc_desc = processor_utils.ProcessorDesc([fetch_unit], [FuncUnit(writeback_unit, [memory_unit])], [], itertools.starmap(FuncUnit, [[memory_unit, [execute_unit]], [execute_unit, [decode_unit]], [decode_unit, [fetch_unit]]]))
hw_desc = hw_loading.HwDesc(proc_desc, processor_utils.load_isa([("LW", "MEM"), ("ADD", "ALU")], processor_utils.get_abilities(proc_desc)))

In [ ]:
prog = program_utils.compile_program(program_utils.read_program(["LW R1, (R2)", "ADD R3, R4, R5", "ADD R6, R7, R8", "ADD R9, R10, R11"]), hw_desc.isa)

In [ ]:
pprint.pp(sim_services.simulate(prog, sim_services.HwSpec(hw_desc.processor)))